In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import datetime

# 우리나라 기후 불러오기

In [3]:
# 파일 불러오기
df = pd.read_csv('./국내기후(백령도제외).csv', index_col=0)

# 시간 datetim형식으로 바꾸기
df['시간'] = pd.to_datetime(df['시간'], format='%Y-%m-%d')

# 2006 - 2021-08-31의 기후 데이터만 가져오기
df_k = df[df['시간'] > '2006-12-09']
df_k = df_k[df_k['시간'] < '2021-09-01']
df_k = df_k.reset_index()
df_k.drop('index', axis=1, inplace=True)

# 황사예측에 필요없는 기후데이터 drop
df_seoul = df_k[df_k.columns]
df_seoul.columns
df_seoul.drop(['1시간 최다강수량','1시간 최다 일사량', '합계 일조 시간','합계 일사','일 최심신적설','일 최심적설',
               '합계 3시간 신적설',
             '합계 대형증발량','합계 소형증발량'], axis=1, inplace=True)

# column 이름 바꾸기
df_seoul.rename(columns={'시간':'일시', '관측지점':'도시명'}, inplace=True)

# 우리나라를 동부, 서부, 남부지역으로 군집
west_w = ['서울','전주','광주']
east_w = ['강릉','춘천','청주','대구']
south_w = ['제주']

no_li_w=[]
for i in df_seoul['도시명']:
    if i in west_w:
        no_li_w.append('ko_w')
    elif i in east_w:
        no_li_w.append('ko_e')
    else:
        no_li_w.append('ko_s')
        
df_seoul['군집'] = no_li_w

# 군집별로 같은 날짜로 묶어주기
df_seoul_n = df_seoul.groupby(['군집', '일시']).agg({'평균 기온':'mean','최저 기온':'min','최고 기온':'max',
                                                '일강수량':'mean','최대 순간풍속': 'mean','최대 순간 풍속 풍향':'mean',
                                               '최대 풍속':'mean','최대 풍속 풍향':'mean','평균 풍속':'mean','풍정합':'mean',
                                                '최대 풍향':'mean',
                                               '평균 이슬점온도':'mean','평균 상대습도':'mean','평균 증기압':'mean',
                                               '평균 현지기압':'mean','최고 해면 기압':'max','최저 해면기압':'min',
                                               '평균 해면기압':'mean',
                                               '평균 전운량':'mean','평균 중하층운량':'mean','평균 지면온도':'mean',
                                               '안개 계속 시간':'mean'})
df_seoul_n = df_seoul_n.reset_index()

# 각 column마다 군집별 이름 넣어주기
tdf_k = pd.DataFrame()

gun_k_li = list(df_seoul_n['군집'].unique())

for gun in gun_k_li:
    tmp_k = df_seoul_n[df_seoul_n['군집']==gun].drop(['군집','일시'],axis=1).reset_index(drop=True)
    tmp_k.columns = gun+tmp_k.columns
    tdf_k = pd.concat([tdf_k,tmp_k],axis=1)
df_k_g = tdf_k
df_k_g

,ko_e평균 기온,ko_e최저 기온,ko_e최고 기온,ko_e일강수량,ko_e최대 순간풍속,ko_e최대 순간 풍속 풍향,ko_e최대 풍속,ko_e최대 풍속 풍향,ko_e평균 풍속,ko_e풍정합,...,ko_w평균 상대습도,ko_w평균 증기압,ko_w평균 현지기압,ko_w최고 해면 기압,ko_w최저 해면기압,ko_w평균 해면기압,ko_w평균 전운량,ko_w평균 중하층운량,ko_w평균 지면온도,ko_w안개 계속 시간
0,1.550,-4.1,8.1,0.000,6.550,267.5,3.975,277.5,1.950,1684.50,...,59.200000,3.866667,1019.333333,1030.0,1025.9,1028.333333,2.366667,0.400000,2.333333,0.000000
1,1.675,-6.9,9.2,0.000,4.975,197.5,3.225,220.0,1.425,1231.00,...,58.566667,4.266667,1018.000000,1030.0,1024.1,1026.900000,4.666667,1.366667,2.866667,0.000000
2,4.675,-3.3,13.1,0.000,6.875,235.0,3.975,247.5,1.575,1360.75,...,65.466667,6.000000,1016.666667,1027.2,1023.0,1025.466667,3.266667,1.966667,4.233333,0.000000
3,5.725,-2.0,11.5,0.000,7.500,272.5,3.875,267.5,1.725,1490.25,...,75.233333,7.533333,1014.533333,1025.6,1020.8,1023.300000,6.933333,4.666667,6.000000,0.000000
4,4.050,-2.8,9.1,0.000,6.125,280.0,4.025,277.5,1.475,1274.25,...,66.333333,5.666667,1012.933333,1024.3,1019.4,1021.766667,5.700000,3.266667,3.900000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5374,23.200,19.8,28.7,8.875,6.350,280.0,3.900,297.5,1.075,945.00,...,92.233333,25.266667,1004.433333,1014.4,1010.3,1012.766667,8.766667,5.566667,24.333333,0.166667
5375,24.575,19.3,31.9,0.075,5.525,192.5,3.350,192.5,1.450,1249.50,...,82.166667,24.300000,1005.633333,1015.5,1012.3,1013.866667,7.200000,4.833333,25.933333,0.000000
5376,23.675,18.6,30.6,6.600,5.725,225.0,3.475,220.0,1.400,1191.00,...,89.300000,26.600000,1005.133333,1015.5,1012.0,1013.366667,8.866667,5.433333,26.066667,0.000000
5377,24.200,19.2,30.6,0.050,4.500,165.0,2.750,217.5,1.250,1054.00,...,83.700000,25.700000,1007.200000,1016.8,1013.7,1015.433333,8.433333,5.000000,26.100000,0.000000


# 우리나라 고층기후 데이터

In [5]:
# 고층 데이터 불러오기
df_k_h = pd.read_csv('./preprocessed_고층기후데이터.csv', index_col=0)
df_k_h = df_k_h.reset_index()

# 1995 - 2021-08-31의 기후 데이터만 가져오기
df_k_h['일시'] = pd.to_datetime(df_k_h['일시'], format='%Y-%m-%d')
df_k_h = df_k_h[df_k_h['일시'] > '2006-12-09']
df_k_h = df_k_h[df_k_h['일시'] < '2021-09-01']
df_k_h = df_k_h.reset_index()
df_k_h.drop('index', axis=1, inplace=True)

df_k_h

,일시,50hPa_고도,50hPa_기온,50hPa_이슬점,50hPa_풍향,50hPa_풍속,70hPa_고도,70hPa_기온,70hPa_이슬점,70hPa_풍향,...,925hPa_고도,925hPa_기온,925hPa_이슬점,925hPa_풍향,925hPa_풍속,1000hPa_고도,1000hPa_기온,1000hPa_이슬점,1000hPa_풍향,1000hPa_풍속
0,2006-12-10,20405.000000,-57.500,-67.250000,258.750000,53.000000,18300.000000,-58.650000,-68.150000,262.500000,...,839.00,-4.100000,-20.600000,237.500000,10.000000,220.500000,-0.600,-12.600000,261.25,5.750000
1,2006-12-11,20377.500000,-57.325,-69.075000,260.000000,47.500000,18280.000000,-60.475000,-71.850000,258.750000,...,829.25,-1.700000,-9.500000,153.750000,11.000000,205.500000,1.625,-8.625000,117.50,5.750000
2,2006-12-12,20452.500000,-59.150,-73.150000,260.000000,37.250000,18342.500000,-60.550000,-74.300000,260.000000,...,826.00,1.750000,-3.300000,266.250000,12.000000,193.250000,5.700,-0.050000,157.50,5.000000
3,2006-12-13,20492.500000,-58.575,-78.450000,256.666667,38.666667,18385.000000,-63.050000,-80.050000,260.000000,...,811.00,1.375000,-6.825000,296.250000,11.250000,178.250000,5.800,-1.600000,270.00,7.750000
4,2006-12-14,20477.500000,-58.600,-72.350000,258.333333,50.583333,18390.000000,-63.650000,-76.400000,258.125000,...,793.25,-0.775000,-5.525000,288.750000,11.500000,166.000000,3.600,-5.300000,247.50,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5374,2021-08-27,20969.583333,-57.950,-70.825000,67.500000,9.708333,18878.750000,-65.283333,-73.658333,259.583333,...,790.50,17.600000,15.700000,338.750000,12.750000,117.750000,21.050,20.000000,232.50,4.250000
5375,2021-08-28,20968.750000,-58.750,-74.125000,124.375000,11.375000,18865.000000,-64.400000,-74.150000,247.500000,...,803.50,18.150000,14.775000,302.500000,8.000000,128.500000,21.300,18.250000,203.75,5.750000
5376,2021-08-29,20958.750000,-58.200,-81.200000,113.125000,6.875000,18861.250000,-64.600000,-79.850000,260.000000,...,806.00,18.200000,15.025000,178.750000,8.250000,131.250000,21.850,19.800000,163.75,3.750000
5377,2021-08-30,20970.000000,-57.825,-74.700000,142.500000,9.625000,18868.750000,-63.175000,-73.925000,251.250000,...,821.50,19.750000,13.500000,261.250000,4.500000,144.500000,22.650,19.875000,258.75,3.750000


# 주변국 데이터 불러오기

In [4]:
# 주변국 기후데이터 가져오기
df_w = pd.read_csv('./preprocessed_주변국 기상데이터_199601to202108.csv', index_col=0)

# 2006년 이후 데이터만 가져오기
df_w['일시'] = pd.to_datetime(df_w['일시'], format='%Y-%m-%d')
df_w = df_w[df_w['일시'] > '2006-12-09']
df_w = df_w.reset_index()
df_w.drop('index', axis=1, inplace=True)

# 같은 날짜에 군집한 지역마다 기후 묶어주기
df_ww = df_w.groupby(['군집', '일시']).agg({ '습도':'mean','풍향':'mean', 
                                            '풍속':'mean', '강수량':'mean','해면기압':'mean', '이슬점 온도':'mean'})
df_ww = df_ww.reset_index()

# 군집한 지역의 기후마다 이름 붙여주기 
tdf = pd.DataFrame({'일시':df_ww['일시'].unique()})

gunli = list(df_ww['군집'].unique())

for gun in gunli:
    tmp = df_ww[df_ww['군집']==gun].drop('군집',axis=1)
    tmp.columns = ['일시', f'{gun}습도', f'{gun}풍향', f'{gun}풍속', f'{gun}강수량', f'{gun}해면기압', f'{gun}이슬점 온도']
    tdf = pd.merge(tdf,tmp, how='left',on='일시')
df_w_w = tdf
df_w_w

,일시,n1습도,n1풍향,n1풍속,n1강수량,n1해면기압,n1이슬점 온도,n2습도,n2풍향,n2풍속,...,n6풍속,n6강수량,n6해면기압,n6이슬점 온도,n7습도,n7풍향,n7풍속,n7강수량,n7해면기압,n7이슬점 온도
0,2006-12-10,56.928571,230.000000,1.160714,0.000000,1028.351488,-9.167857,69.571429,27.5,0.607143,...,2.017857,0.000000,1025.631786,-7.886429,64.904167,110.000000,2.041667,0.000000,1028.425000,-1.791667
1,2006-12-11,65.468750,202.500000,2.062500,0.445600,1026.965625,-4.284375,77.772321,80.0,0.281250,...,3.010714,0.000000,1021.521429,-3.608929,76.262500,263.333333,2.500000,-0.047822,1024.695833,1.304167
2,2006-12-12,72.770833,179.274917,1.005952,0.000000,1024.439583,-0.572024,73.654762,40.0,0.285714,...,2.364286,0.000000,1023.791071,-4.638214,79.154167,323.333333,1.791667,0.133333,1025.512500,3.220833
3,2006-12-13,62.916667,248.750000,1.416667,0.000000,1023.255208,-3.912500,55.000000,102.5,1.750000,...,2.282143,0.000000,1024.350714,-11.458214,71.641667,220.000000,1.666667,1.000000,1025.920833,1.837500
4,2006-12-14,60.915179,167.500000,0.805804,0.000000,1021.968527,-6.006473,66.916667,115.0,0.531250,...,2.389286,0.000000,1019.422857,-9.447500,68.450000,170.000000,1.875000,0.000000,1022.854167,-0.191667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5374,2021-08-27,82.468750,111.250000,0.906250,0.880792,1011.759375,20.028125,80.593750,350.0,0.937500,...,2.800000,0.000000,1009.500000,16.523214,78.830556,240.000000,2.125000,-0.047822,1012.870833,21.366667
5375,2021-08-28,77.921875,115.000000,1.500000,0.000000,1013.007812,19.660938,75.125000,195.0,1.375000,...,1.996429,-0.594099,1011.968571,16.480000,81.301190,106.666667,1.404762,-0.739609,1012.662500,22.993452
5376,2021-08-29,83.125000,162.500000,1.031250,1.669771,1013.484375,19.915625,77.875000,42.5,1.218750,...,1.982143,-0.915458,1013.549643,17.045000,87.795833,210.000000,2.083333,-0.702848,1013.058333,23.450000
5377,2021-08-30,77.578125,183.750000,0.937500,0.000000,1015.429688,19.173437,73.281250,157.5,1.406250,...,1.510714,-0.915262,1015.932500,17.566429,83.704167,220.000000,1.916667,-0.625120,1012.554167,24.179167


# 황사데이터

In [5]:
# 황사데이터 가져오기
df_h = pd.read_csv('./황사.csv', encoding='euckr')

# 2006 - 2021-08-31의 기후 데이터만 가져오기
df_h['일시'] = pd.to_datetime(df_h['일시'], format='%Y-%m-%d')
df_h = df_h[df_h['일시'] > '2006-12-09']
df_h = df_h[df_h['일시'] < '2021-09-01']
df_h = df_h.reset_index()
df_h.drop('index', axis=1, inplace=True)

# 동부, 서부, 남부지역 군집
# 서부지역 
west_li = ['서울', '강화', '수원', '안면도(감)', '군산', '광주', '광덕산', '백령도', '관악산', '오성산', 
           '전주', '북격렬비도', '천안', '연평도']
# 동부지역
east_li = ['춘천', '추풍령', '안동', '대구', '속초', '북춘천', '대관령', '춘천', '울릉도독도(감)', '영월', '울진', '문경', '영덕']

# 남부지역
south_li = ['고산', '진도군', '구덕산', '울산', '흑산도', '진도(첨찰산)', '진주']

no_li=[]
for i in df_h['지점명']:
    if i in west_li:
        no_li.append('d1')
    elif i in east_li:
        no_li.append('d2')
    else:
        no_li.append('d3')
        
df_h['군집'] = no_li

# 필요없는 column drop
df_h.drop('지점', axis=1, inplace=True)
df_h.rename({'일 미세먼지 농도(㎍/㎥)' : '농도'}, axis=1, inplace=True)

df_h_test = df_h[df_h.columns]

# 군집별 날짜마다 평균으로 묶어주기
df_h_test = df_h_test.groupby(['군집', '일시']).agg({ '농도':'mean'})
df_h_test = df_h_test.reset_index()

# 미세먼지 농도마다 군집번호 column에 넣기
tdf_h = pd.DataFrame({'일시':df_h_test['일시'].unique()})

gun_li_h = list(df_h_test['군집'].unique())

for gun in gun_li_h:
    tmp_h = df_h_test[df_h_test['군집']==gun].drop('군집',axis=1)
    tmp_h.columns = ['일시', f'{gun}농도']
    tdf_h = pd.merge(tdf_h,tmp_h, how='left',on='일시')
df_h_n = tdf_h

import datetime as dt

# 2006-2021까지 비어있는 날들 보간으로 채워주기
dat_a = pd.DataFrame({'일시':np.arange(dt.datetime(2006,12,10),dt.datetime(2021,9, 1),dt.timedelta(days=1))})
dat_a['일시'] = dat_a['일시'].apply(lambda x : x.strftime('%Y-%m-%d'))

dat_a['일시'] = dat_a['일시'].astype('datetime64')

df_h_test = pd.merge(dat_a, df_h_n, how='left', on='일시')

df_h_test['d1농도'].interpolate(inplace=True)
df_h_test['d2농도'].interpolate(inplace=True)
df_h_test['d3농도'].interpolate(inplace=True)

df_h_test_a = df_h_test[df_h_test.columns]
df_h_test_w = df_h_test[df_h_test.columns]

df_h_test_a

,일시,d1농도,d2농도,d3농도
0,2006-12-10,24.555556,21.000000,23.000000
1,2006-12-11,38.111111,20.600000,17.333333
2,2006-12-12,63.444444,42.166667,26.666667
3,2006-12-13,95.777778,78.500000,43.333333
4,2006-12-14,64.666667,70.833333,60.000000
...,...,...,...,...
5374,2021-08-27,29.000000,32.200000,22.166667
5375,2021-08-28,30.272727,22.900000,16.250000
5376,2021-08-29,24.555556,23.200000,19.250000
5377,2021-08-30,35.363636,27.200000,19.400000


In [9]:
# 전국 미세농도 데이터
# 미세농도가 나쁨기준(80)은 체크해주기
check_li = [] 
for h in range(len(df_h_test_a)):
    if (df_h_test_a.iloc[h,1] > 80):
        check_li.append(1)
    elif (df_h_test_a.iloc[h,2] > 80):
        check_li.append(1)
    elif (df_h_test_a.iloc[h,3] > 80):
        check_li.append(1)
    else:
        check_li.append(0)
        
df_h_test_a['check'] = check_li

# 황사발생 3일전 체크
df_h_test_a['target'] = df_h_test_a.check.shift(-3)


df_h_test_a['target'].fillna(0, inplace=True)
df_h_test_a['target'] = df_h_test_a['target'].astype(int)

df_h_test_f = df_h_test_a
df_h_test_f.drop(['d1농도','d2농도','d3농도'], axis=1, inplace=True)
df_h_test_f

,일시,check,target
0,2006-12-10,0,1
1,2006-12-11,0,0
2,2006-12-12,0,1
3,2006-12-13,1,0
4,2006-12-14,0,0
...,...,...,...
5374,2021-08-27,0,0
5375,2021-08-28,0,0
5376,2021-08-29,0,0
5377,2021-08-30,0,0


In [10]:
# 서부 미세농도 데이터
# 미세농도가 나쁨기준(80)은 체크해주기
df_h_test_w = df_h_test[['일시','d1농도']]

df_h_test_w['check'] = df_h_test_w['d1농도'].apply(lambda x: 1 if x > 80 else 0)
df_h_test_w

# 황사발생 3일전 체크
df_h_test_w['target'] = df_h_test_w.check.shift(-2)
df_h_test_w['target'].fillna(0, inplace=True)
df_h_test_w['target'] = df_h_test_w['target'].astype(int)

df_h_test_w_a = df_h_test_w.drop('d1농도', axis=1)
df_h_test_w_a

,일시,check,target
0,2006-12-10,0,0
1,2006-12-11,0,1
2,2006-12-12,0,0
3,2006-12-13,1,1
4,2006-12-14,0,0
...,...,...,...
5374,2021-08-27,0,0
5375,2021-08-28,0,0
5376,2021-08-29,0,0
5377,2021-08-30,0,0


# merge

In [11]:
# 1 우리나라+고층+주변국+황사
df_mm = pd.concat([df_w_w,df_k_g ], axis=1)
df_mm_mm = pd.merge(df_mm,df_h_test_f, how='left', on='일시')

df_a = df_mm_mm[df_mm_mm['일시'] < '2019-01-01']
df_b = df_mm_mm[df_mm_mm['일시'] > '2019-12-31']

In [12]:
# 2 우리나라+고층+주변국+황사서쪽
df_mm = pd.concat([df_w_w,df_k_g ], axis=1)
df_mm_mm = pd.merge(df_mm,df_h_test_w_a, how='left', on='일시')

df_a = df_mm_mm[df_mm_mm['일시'] < '2019-01-01']
df_b = df_mm_mm[df_mm_mm['일시'] > '2019-12-31']

In [13]:
df_ml_m = pd.concat([df_a,df_b])

In [14]:
df_ml_m

,일시,n1습도,n1풍향,n1풍속,n1강수량,n1해면기압,n1이슬점 온도,n2습도,n2풍향,n2풍속,...,ko_w평균 현지기압,ko_w최고 해면 기압,ko_w최저 해면기압,ko_w평균 해면기압,ko_w평균 전운량,ko_w평균 중하층운량,ko_w평균 지면온도,ko_w안개 계속 시간,check,target
0,2006-12-10,56.928571,230.000000,1.160714,0.000000,1028.351488,-9.167857,69.571429,27.5,0.607143,...,1019.333333,1030.0,1025.9,1028.333333,2.366667,0.400000,2.333333,0.000000,0,0
1,2006-12-11,65.468750,202.500000,2.062500,0.445600,1026.965625,-4.284375,77.772321,80.0,0.281250,...,1018.000000,1030.0,1024.1,1026.900000,4.666667,1.366667,2.866667,0.000000,0,1
2,2006-12-12,72.770833,179.274917,1.005952,0.000000,1024.439583,-0.572024,73.654762,40.0,0.285714,...,1016.666667,1027.2,1023.0,1025.466667,3.266667,1.966667,4.233333,0.000000,0,0
3,2006-12-13,62.916667,248.750000,1.416667,0.000000,1023.255208,-3.912500,55.000000,102.5,1.750000,...,1014.533333,1025.6,1020.8,1023.300000,6.933333,4.666667,6.000000,0.000000,1,1
4,2006-12-14,60.915179,167.500000,0.805804,0.000000,1021.968527,-6.006473,66.916667,115.0,0.531250,...,1012.933333,1024.3,1019.4,1021.766667,5.700000,3.266667,3.900000,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5374,2021-08-27,82.468750,111.250000,0.906250,0.880792,1011.759375,20.028125,80.593750,350.0,0.937500,...,1004.433333,1014.4,1010.3,1012.766667,8.766667,5.566667,24.333333,0.166667,0,0
5375,2021-08-28,77.921875,115.000000,1.500000,0.000000,1013.007812,19.660938,75.125000,195.0,1.375000,...,1005.633333,1015.5,1012.3,1013.866667,7.200000,4.833333,25.933333,0.000000,0,0
5376,2021-08-29,83.125000,162.500000,1.031250,1.669771,1013.484375,19.915625,77.875000,42.5,1.218750,...,1005.133333,1015.5,1012.0,1013.366667,8.866667,5.433333,26.066667,0.000000,0,0
5377,2021-08-30,77.578125,183.750000,0.937500,0.000000,1015.429688,19.173437,73.281250,157.5,1.406250,...,1007.200000,1016.8,1013.7,1015.433333,8.433333,5.000000,26.100000,0.000000,0,0


# 모델링

In [20]:
y = df_ml_m['target']
x = df_ml_m.drop(['일시','check','target'], axis=1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier

In [22]:
from sklearn.metrics import classification_report, roc_auc_score

In [40]:
from sklearn.model_selection import GridSearchCV

In [44]:
from catboost import CatBoostClassifier

In [23]:
lr = LogisticRegression()
svc = SVC(probability=True)
rfc = RandomForestClassifier()
xg = XGBClassifier()
knn = KNeighborsClassifier()
lgbm = LGBMClassifier()

models = [lr, svc, rfc, xg, knn, lgbm]

In [24]:
for m in models:
    m.fit(x_train, y_train)
    pred = m.predict(x_test)
    preb = m.predict_proba(x_test)
    
    print(classification_report(y_test, pred))
    print('-'*50)
    print(roc_auc_score(y_test, preb[:,1]))
    print('='*100)
    

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       942
           1       0.00      0.00      0.00        61

    accuracy                           0.94      1003
   macro avg       0.47      0.50      0.48      1003
weighted avg       0.88      0.94      0.91      1003

--------------------------------------------------
0.7822735024886012
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       942
           1       0.00      0.00      0.00        61

    accuracy                           0.94      1003
   macro avg       0.47      0.50      0.48      1003
weighted avg       0.88      0.94      0.91      1003

--------------------------------------------------
0.7122098082210851
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       942
           1       0.00      0.00      0.00        61

    accuracy                           0.9

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [26]:
kfold = KFold()
for m in models:
    score = cross_val_score(m, x_train, y_train, cv=kfold)
    
    print(score)
    print(np.mean(score))
    print(np.std(score))
    print('='*100)

[0.94894147 0.93765586 0.94264339 0.94887781 0.94763092]
0.945149889907858
0.004401635313264614
[0.94894147 0.93640898 0.94264339 0.95012469 0.94763092]
0.945149889907858
0.00505897818033148
[0.94894147 0.93640898 0.94139651 0.95012469 0.94887781]
0.945149889907858
0.00535748948931456
[22:47:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:47:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:47:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.c

In [27]:
pd.Series(xg.feature_importances_, x.columns).sort_values(ascending=False).head(60)

ko_s평균 증기압         0.050253
ko_w평균 현지기압        0.038362
ko_s평균 해면기압        0.028099
ko_s평균 기온          0.025065
ko_w평균 증기압         0.021233
ko_w최저 기온          0.021116
ko_e안개 계속 시간       0.018010
n1이슬점 온도           0.017311
n4습도               0.015897
ko_e최저 기온          0.015615
n4이슬점 온도           0.015362
n3풍향               0.013523
n5이슬점 온도           0.013439
ko_s최대 풍속 풍향       0.013346
n6이슬점 온도           0.013287
ko_w평균 풍속          0.013255
ko_s최대 풍속          0.013220
n7이슬점 온도           0.012954
ko_w안개 계속 시간       0.012691
ko_w최저 해면기압        0.012394
ko_e평균 전운량         0.012359
n7해면기압             0.012109
n6해면기압             0.011820
ko_e최대 순간풍속        0.011621
ko_s최고 해면 기압       0.011318
ko_w최대 풍속 풍향       0.011312
ko_e평균 중하층운량       0.011057
ko_w최대 순간풍속        0.011053
n6강수량              0.010883
ko_s평균 중하층운량       0.010751
ko_s최고 기온          0.010739
n7풍속               0.010605
ko_s최대 순간 풍속 풍향    0.010481
n6습도               0.010276
ko_e평균 상대습도        0.010155
n5해면기압             0

In [28]:
# Scaler 적용

from sklearn.preprocessing import StandardScaler
std = StandardScaler()

std.fit(x_train)
x_train_std = std.transform(x_train)
x_test_std = std.transform(x_test)

In [29]:
models = [lr, svc, rfc, xg, knn, lgbm]

for m in models:
    m.fit(x_train_std, y_train)
    pred = m.predict(x_test_std)
    preb = m.predict_proba(x_test_std)
    
    print(classification_report(y_test, pred))
    print('-'*50)
    print(roc_auc_score(y_test, preb[:,1]))
    print('='*100)
    

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       942
           1       0.43      0.05      0.09        61

    accuracy                           0.94      1003
   macro avg       0.69      0.52      0.53      1003
weighted avg       0.91      0.94      0.91      1003

--------------------------------------------------
0.8398071769169191
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       942
           1       0.00      0.00      0.00        61

    accuracy                           0.94      1003
   macro avg       0.47      0.50      0.48      1003
weighted avg       0.88      0.94      0.91      1003

--------------------------------------------------
0.7729455988305314
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       942
           1       0.00      0.00      0.00        61

    accuracy                           0.9

In [30]:
pd.Series(xg.feature_importances_, x.columns).sort_values(ascending=False).head(60)

ko_s평균 증기압         0.050253
ko_w평균 현지기압        0.038362
ko_s평균 해면기압        0.028099
ko_s평균 기온          0.025065
ko_w평균 증기압         0.021233
ko_w최저 기온          0.021116
ko_e안개 계속 시간       0.018010
n1이슬점 온도           0.017311
n4습도               0.015897
ko_e최저 기온          0.015615
n4이슬점 온도           0.015362
n3풍향               0.013523
n5이슬점 온도           0.013439
ko_s최대 풍속 풍향       0.013346
n6이슬점 온도           0.013287
ko_w평균 풍속          0.013255
ko_s최대 풍속          0.013220
n7이슬점 온도           0.012954
ko_w안개 계속 시간       0.012691
ko_w최저 해면기압        0.012394
ko_e평균 전운량         0.012359
n7해면기압             0.012109
n6해면기압             0.011820
ko_e최대 순간풍속        0.011621
ko_s최고 해면 기압       0.011318
ko_w최대 풍속 풍향       0.011312
ko_e평균 중하층운량       0.011057
ko_w최대 순간풍속        0.011053
n6강수량              0.010883
ko_s평균 중하층운량       0.010751
ko_s최고 기온          0.010739
n7풍속               0.010605
ko_s최대 순간 풍속 풍향    0.010481
n6습도               0.010276
ko_e평균 상대습도        0.010155
n5해면기압             0

In [31]:
# Smote 적용해서 imbalance한 데이터 맞춰주기
import imblearn
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=100)
x_train_std_sm, y_train_sm = smote.fit_resample(x_train_std, y_train)

y_train_sm.value_counts()

0    3791
1    3791
Name: target, dtype: int64

In [32]:
for m in models:
    m.fit(x_train_std_sm, y_train_sm)
    pred = m.predict(x_test_std)
    preb = m.predict_proba(x_test_std)
    
    print(classification_report(y_test, pred))
    print('-'*50)
    print(roc_auc_score(y_test, preb[:,1]))
    print('='*100)
    

              precision    recall  f1-score   support

           0       0.98      0.80      0.88       942
           1       0.19      0.72      0.30        61

    accuracy                           0.80      1003
   macro avg       0.59      0.76      0.59      1003
weighted avg       0.93      0.80      0.85      1003

--------------------------------------------------
0.8397897741115867
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       942
           1       0.26      0.34      0.29        61

    accuracy                           0.90      1003
   macro avg       0.61      0.64      0.62      1003
weighted avg       0.91      0.90      0.91      1003

--------------------------------------------------
0.8059761233510843
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       942
           1       0.20      0.08      0.12        61

    accuracy                           0.9

In [51]:
# catboost 모델링
cat = CatBoostClassifier()

cat.fit(x_train_std_sm, y_train_sm)

preds = cat.predict(x_test_std)
print(classification_report(y_test, preds))

Learning rate set to 0.024468
0:	learn: 0.6809559	total: 27.7ms	remaining: 27.7s
1:	learn: 0.6671171	total: 50.9ms	remaining: 25.4s
2:	learn: 0.6554384	total: 75.5ms	remaining: 25.1s
3:	learn: 0.6449512	total: 99.7ms	remaining: 24.8s
4:	learn: 0.6338601	total: 123ms	remaining: 24.5s
5:	learn: 0.6236207	total: 144ms	remaining: 23.9s
6:	learn: 0.6134122	total: 164ms	remaining: 23.3s
7:	learn: 0.6041880	total: 184ms	remaining: 22.8s
8:	learn: 0.5958965	total: 207ms	remaining: 22.8s
9:	learn: 0.5878846	total: 229ms	remaining: 22.7s
10:	learn: 0.5808564	total: 250ms	remaining: 22.5s
11:	learn: 0.5730684	total: 271ms	remaining: 22.3s
12:	learn: 0.5656718	total: 291ms	remaining: 22.1s
13:	learn: 0.5587651	total: 311ms	remaining: 21.9s
14:	learn: 0.5529487	total: 331ms	remaining: 21.7s
15:	learn: 0.5470585	total: 352ms	remaining: 21.6s
16:	learn: 0.5408546	total: 372ms	remaining: 21.5s
17:	learn: 0.5354739	total: 392ms	remaining: 21.4s
18:	learn: 0.5293640	total: 417ms	remaining: 21.5s
19:	lea

167:	learn: 0.2209823	total: 3.5s	remaining: 17.3s
168:	learn: 0.2202830	total: 3.52s	remaining: 17.3s
169:	learn: 0.2192475	total: 3.54s	remaining: 17.3s
170:	learn: 0.2186497	total: 3.56s	remaining: 17.3s
171:	learn: 0.2179102	total: 3.58s	remaining: 17.2s
172:	learn: 0.2172499	total: 3.6s	remaining: 17.2s
173:	learn: 0.2163084	total: 3.62s	remaining: 17.2s
174:	learn: 0.2156737	total: 3.64s	remaining: 17.1s
175:	learn: 0.2146987	total: 3.66s	remaining: 17.1s
176:	learn: 0.2139126	total: 3.68s	remaining: 17.1s
177:	learn: 0.2131956	total: 3.7s	remaining: 17.1s
178:	learn: 0.2125726	total: 3.72s	remaining: 17.1s
179:	learn: 0.2120712	total: 3.74s	remaining: 17s
180:	learn: 0.2113710	total: 3.76s	remaining: 17s
181:	learn: 0.2106653	total: 3.78s	remaining: 17s
182:	learn: 0.2098028	total: 3.8s	remaining: 17s
183:	learn: 0.2093879	total: 3.82s	remaining: 17s
184:	learn: 0.2085876	total: 3.84s	remaining: 16.9s
185:	learn: 0.2078065	total: 3.86s	remaining: 16.9s
186:	learn: 0.2071966	tota

330:	learn: 0.1293618	total: 6.78s	remaining: 13.7s
331:	learn: 0.1289299	total: 6.8s	remaining: 13.7s
332:	learn: 0.1285027	total: 6.82s	remaining: 13.7s
333:	learn: 0.1281349	total: 6.84s	remaining: 13.6s
334:	learn: 0.1277113	total: 6.86s	remaining: 13.6s
335:	learn: 0.1274304	total: 6.88s	remaining: 13.6s
336:	learn: 0.1270720	total: 6.9s	remaining: 13.6s
337:	learn: 0.1266951	total: 6.92s	remaining: 13.6s
338:	learn: 0.1260471	total: 6.94s	remaining: 13.5s
339:	learn: 0.1258250	total: 6.96s	remaining: 13.5s
340:	learn: 0.1254308	total: 6.99s	remaining: 13.5s
341:	learn: 0.1251371	total: 7s	remaining: 13.5s
342:	learn: 0.1247553	total: 7.03s	remaining: 13.5s
343:	learn: 0.1244828	total: 7.05s	remaining: 13.4s
344:	learn: 0.1241905	total: 7.07s	remaining: 13.4s
345:	learn: 0.1238997	total: 7.09s	remaining: 13.4s
346:	learn: 0.1237038	total: 7.11s	remaining: 13.4s
347:	learn: 0.1233684	total: 7.13s	remaining: 13.4s
348:	learn: 0.1230125	total: 7.15s	remaining: 13.3s
349:	learn: 0.122

497:	learn: 0.0761633	total: 10.3s	remaining: 10.4s
498:	learn: 0.0758423	total: 10.3s	remaining: 10.4s
499:	learn: 0.0756405	total: 10.4s	remaining: 10.4s
500:	learn: 0.0751362	total: 10.4s	remaining: 10.3s
501:	learn: 0.0749310	total: 10.4s	remaining: 10.3s
502:	learn: 0.0747630	total: 10.4s	remaining: 10.3s
503:	learn: 0.0746054	total: 10.4s	remaining: 10.3s
504:	learn: 0.0744727	total: 10.5s	remaining: 10.3s
505:	learn: 0.0742565	total: 10.5s	remaining: 10.2s
506:	learn: 0.0740495	total: 10.5s	remaining: 10.2s
507:	learn: 0.0738843	total: 10.5s	remaining: 10.2s
508:	learn: 0.0736791	total: 10.5s	remaining: 10.2s
509:	learn: 0.0734690	total: 10.6s	remaining: 10.1s
510:	learn: 0.0732769	total: 10.6s	remaining: 10.1s
511:	learn: 0.0731485	total: 10.6s	remaining: 10.1s
512:	learn: 0.0729443	total: 10.6s	remaining: 10.1s
513:	learn: 0.0727418	total: 10.6s	remaining: 10.1s
514:	learn: 0.0725706	total: 10.7s	remaining: 10s
515:	learn: 0.0723722	total: 10.7s	remaining: 10s
516:	learn: 0.07

663:	learn: 0.0498389	total: 14.6s	remaining: 7.37s
664:	learn: 0.0497419	total: 14.6s	remaining: 7.36s
665:	learn: 0.0495596	total: 14.6s	remaining: 7.33s
666:	learn: 0.0494268	total: 14.7s	remaining: 7.32s
667:	learn: 0.0492938	total: 14.7s	remaining: 7.3s
668:	learn: 0.0491562	total: 14.7s	remaining: 7.28s
669:	learn: 0.0489947	total: 14.7s	remaining: 7.26s
670:	learn: 0.0489148	total: 14.8s	remaining: 7.24s
671:	learn: 0.0487946	total: 14.8s	remaining: 7.22s
672:	learn: 0.0486910	total: 14.8s	remaining: 7.2s
673:	learn: 0.0485997	total: 14.8s	remaining: 7.18s
674:	learn: 0.0484752	total: 14.9s	remaining: 7.16s
675:	learn: 0.0483522	total: 14.9s	remaining: 7.14s
676:	learn: 0.0482233	total: 14.9s	remaining: 7.12s
677:	learn: 0.0480595	total: 15s	remaining: 7.1s
678:	learn: 0.0479335	total: 15s	remaining: 7.08s
679:	learn: 0.0478044	total: 15s	remaining: 7.06s
680:	learn: 0.0477242	total: 15s	remaining: 7.04s
681:	learn: 0.0476540	total: 15.1s	remaining: 7.02s
682:	learn: 0.0475528	t

831:	learn: 0.0337260	total: 18.7s	remaining: 3.78s
832:	learn: 0.0336806	total: 18.7s	remaining: 3.75s
833:	learn: 0.0335894	total: 18.8s	remaining: 3.73s
834:	learn: 0.0335298	total: 18.8s	remaining: 3.71s
835:	learn: 0.0334447	total: 18.8s	remaining: 3.69s
836:	learn: 0.0333863	total: 18.8s	remaining: 3.66s
837:	learn: 0.0333355	total: 18.8s	remaining: 3.64s
838:	learn: 0.0332660	total: 18.9s	remaining: 3.62s
839:	learn: 0.0332060	total: 18.9s	remaining: 3.6s
840:	learn: 0.0331512	total: 18.9s	remaining: 3.57s
841:	learn: 0.0330723	total: 18.9s	remaining: 3.55s
842:	learn: 0.0330250	total: 19s	remaining: 3.53s
843:	learn: 0.0329271	total: 19s	remaining: 3.51s
844:	learn: 0.0328280	total: 19s	remaining: 3.48s
845:	learn: 0.0327595	total: 19s	remaining: 3.46s
846:	learn: 0.0327032	total: 19s	remaining: 3.44s
847:	learn: 0.0326368	total: 19.1s	remaining: 3.42s
848:	learn: 0.0325690	total: 19.1s	remaining: 3.4s
849:	learn: 0.0324949	total: 19.1s	remaining: 3.37s
850:	learn: 0.0324384	to

990:	learn: 0.0240192	total: 22.2s	remaining: 202ms
991:	learn: 0.0239717	total: 22.3s	remaining: 180ms
992:	learn: 0.0239084	total: 22.3s	remaining: 157ms
993:	learn: 0.0238465	total: 22.3s	remaining: 135ms
994:	learn: 0.0237882	total: 22.3s	remaining: 112ms
995:	learn: 0.0237450	total: 22.4s	remaining: 89.8ms
996:	learn: 0.0237182	total: 22.4s	remaining: 67.3ms
997:	learn: 0.0236637	total: 22.4s	remaining: 44.9ms
998:	learn: 0.0236259	total: 22.4s	remaining: 22.4ms
999:	learn: 0.0235725	total: 22.4s	remaining: 0us
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       942
           1       0.27      0.10      0.14        61

    accuracy                           0.93      1003
   macro avg       0.61      0.54      0.55      1003
weighted avg       0.90      0.93      0.91      1003



In [33]:
pd.Series(xg.feature_importances_, x.columns).sort_values(ascending=False).head(60)

n4이슬점 온도           0.102554
ko_s평균 이슬점온도       0.080910
ko_s최대 순간 풍속 풍향    0.028674
ko_s최저 기온          0.027708
ko_e안개 계속 시간       0.026292
ko_w일강수량           0.019546
n4풍향               0.018773
n7강수량              0.017942
n6이슬점 온도           0.017294
ko_e평균 중하층운량       0.017232
n5이슬점 온도           0.017151
ko_s최대 풍향          0.016503
ko_s평균 기온          0.015685
ko_s일강수량           0.015508
n1풍향               0.014900
n1강수량              0.014706
ko_e최대 풍향          0.013411
n6해면기압             0.013110
ko_w안개 계속 시간       0.012712
n4습도               0.012540
n3풍향               0.012521
n7습도               0.012330
ko_s최대 풍속 풍향       0.012145
ko_w평균 이슬점온도       0.011810
n3강수량              0.011586
n5해면기압             0.011431
ko_w평균 중하층운량       0.010307
ko_e평균 풍속          0.009882
n1습도               0.009828
n6습도               0.009812
ko_w최저 해면기압        0.009714
ko_e일강수량           0.009713
n7풍향               0.009562
n1풍속               0.009543
n2풍향               0.009524
n4강수량              0

In [34]:
# Gridsearch 적용
from sklearn.model_selection import GridSearchCV
param = { 'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]}

xg_gv = XGBClassifier()
grid = GridSearchCV(xg_gv, param_grid=param, refit=True, verbose=False)
grid.fit(x_train_std_sm, y_train_sm)

In [35]:
grid.best_params_

In [36]:
grid.best_score_

In [47]:
# gridsearch로 뽑은 best_paramiter를 xgboost에 적용
xg_g = XGBClassifier(colsample_bytree = 0.8,
 gamma = 0.5,
 max_depth = 5,
 min_child_weight = 1,
 subsample = 0.6)

xg_g.fit(x_train_std_sm, y_train_sm)
pred = rfc.predict(x_test_std)
print(classification_report(y_test, pred))

[23:09:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       942
           1       0.20      0.08      0.12        61

    accuracy                           0.92      1003
   macro avg       0.57      0.53      0.54      1003
weighted avg       0.90      0.92      0.91      1003



In [38]:
pd.Series(xg.feature_importances_, x.columns).sort_values(ascending=False).head(60)

n4이슬점 온도           0.102554
ko_s평균 이슬점온도       0.080910
ko_s최대 순간 풍속 풍향    0.028674
ko_s최저 기온          0.027708
ko_e안개 계속 시간       0.026292
ko_w일강수량           0.019546
n4풍향               0.018773
n7강수량              0.017942
n6이슬점 온도           0.017294
ko_e평균 중하층운량       0.017232
n5이슬점 온도           0.017151
ko_s최대 풍향          0.016503
ko_s평균 기온          0.015685
ko_s일강수량           0.015508
n1풍향               0.014900
n1강수량              0.014706
ko_e최대 풍향          0.013411
n6해면기압             0.013110
ko_w안개 계속 시간       0.012712
n4습도               0.012540
n3풍향               0.012521
n7습도               0.012330
ko_s최대 풍속 풍향       0.012145
ko_w평균 이슬점온도       0.011810
n3강수량              0.011586
n5해면기압             0.011431
ko_w평균 중하층운량       0.010307
ko_e평균 풍속          0.009882
n1습도               0.009828
n6습도               0.009812
ko_w최저 해면기압        0.009714
ko_e일강수량           0.009713
n7풍향               0.009562
n1풍속               0.009543
n2풍향               0.009524
n4강수량              0

In [58]:
# gridserch로 catboost 적용
params = {'leaning_rate':[0.01,0.3], 'depth':[3,4]}

cat = CatBoostClassifier()

grid_a = GridSearchCV(cat, param_grid=params, refit=True, verbose= 2, cv=3)

grid_a.fit(x_train_std_sm, y_train_sm)

print(grid.best_params_)
preds = grid.predict(x_test_std)
print(classification_report(y_test, preds))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END .........................depth=3, leaning_rate=0.01; total time=   0.4s
[CV] END .........................depth=3, leaning_rate=0.01; total time=   0.4s
[CV] END .........................depth=3, leaning_rate=0.01; total time=   0.4s
[CV] END ..........................depth=3, leaning_rate=0.3; total time=   0.4s
[CV] END ..........................depth=3, leaning_rate=0.3; total time=   0.4s
[CV] END ..........................depth=3, leaning_rate=0.3; total time=   0.3s
[CV] END .........................depth=4, leaning_rate=0.01; total time=   0.3s
[CV] END .........................depth=4, leaning_rate=0.01; total time=   0.2s
[CV] END .........................depth=4, leaning_rate=0.01; total time=   0.1s
[CV] END ..........................depth=4, leaning_rate=0.3; total time=   0.1s
[CV] END ..........................depth=4, leaning_rate=0.3; total time=   0.1s
[CV] END ..........................depth=4, leani

TypeError: __init__() got an unexpected keyword argument 'leaning_rate'